In [1]:
#Part 3: Advanced Remote Execution Tools
import torch
import syft as sy
hook = sy.TorchHook(torch)

Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was 'C:\ProgramData\Anaconda3\lib\site-packages\tf_encrypted/operations/secure_random/secure_random_module_tf_1.15.2.so'


In [2]:
#Section 3.1 - Pointers to Pointers

bob = sy.VirtualWorker(hook, id='bob')
alice = sy.VirtualWorker(hook, id='alice')

In [3]:
# this is a local tensor
x = torch.tensor([1,2,3,4])
x

tensor([1, 2, 3, 4])

In [4]:

# this sends the local tensor to Bob
x_ptr = x.send(bob)

# this is now a pointer
x_ptr

(Wrapper)>[PointerTensor | me:51423112216 -> bob:82875825576]

In [5]:
# now we can SEND THE POINTER to alice!!!
pointer_to_x_ptr = x_ptr.send(alice)

pointer_to_x_ptr

(Wrapper)>[PointerTensor | me:88294647521 -> alice:51423112216]

In [6]:
# As you can see above, Bob still has the actual data (data is always stored in a LocalTensor type). 
bob._objects

{82875825576: tensor([1, 2, 3, 4])}

In [7]:
# Alice, on the other hand, has x_ptr!! (notice how it points at bob)
alice._objects

{51423112216: (Wrapper)>[PointerTensor | alice:51423112216 -> bob:82875825576]}

In [8]:
# and we can use .get() to get x_ptr back from Alice

x_ptr = pointer_to_x_ptr.get()
x_ptr

(Wrapper)>[PointerTensor | me:51423112216 -> bob:82875825576]

In [9]:
# and then we can use x_ptr to get x back from Bob!

x = x_ptr.get()
x

tensor([1, 2, 3, 4])

In [10]:
bob._objects

{}

In [11]:
alice._objects

{}

In [12]:
p2p2x = torch.tensor([1,2,3,4,5]).send(bob).send(alice)

y = p2p2x + p2p2x

In [13]:
bob._objects

{12770052443: tensor([1, 2, 3, 4, 5]),
 97117778194: tensor([ 2,  4,  6,  8, 10])}

In [14]:
alice._objects

{7271172550: (Wrapper)>[PointerTensor | alice:7271172550 -> bob:12770052443],
 71540644157: (Wrapper)>[PointerTensor | alice:71540644157 -> bob:97117778194]}

In [15]:
y.get().get()

tensor([ 2,  4,  6,  8, 10])

In [16]:
bob._objects

{12770052443: tensor([1, 2, 3, 4, 5])}

In [17]:
alice._objects

{7271172550: (Wrapper)>[PointerTensor | alice:7271172550 -> bob:12770052443]}

In [18]:
p2p2x.get().get()

tensor([1, 2, 3, 4, 5])

In [19]:
bob._objects

{}

In [20]:
alice._objects

{}

In [21]:
#Section 3.2 - Pointer Chain Operations

# x is now a pointer to the data which lives on Bob's machine
x = torch.tensor([1,2,3,4,5]).send(bob)

In [22]:
print('  bob:', bob._objects)
print('alice:',alice._objects)

  bob: {15121922036: tensor([1, 2, 3, 4, 5])}
alice: {}


In [23]:
x = x.move(alice)

In [24]:
print('  bob:', bob._objects)
print('alice:',alice._objects)

  bob: {}
alice: {35336353719: tensor([1, 2, 3, 4, 5])}


In [25]:
x

(Wrapper)>[PointerTensor | me:94491334968 -> alice:35336353719]